In [2]:
# Instalar as dependências necessárias
%pip install boto3 requests fpdf2 urllib3 datetime load_dotenv

import boto3
import urllib3
import requests
from fpdf import FPDF
import json
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/home/palomapatriciafalchi/.local/lib/python3.10/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [3]:

# RODAR QUANDO QUISER NOVAS TRANSCRIÇÕES

# Configurações
load_dotenv('config/.env')
bucket_name = os.getenv('BUCKET_S3')
fireflies_api_key = os.getenv('API_KEY_FIREFLIES')
url = os.getenv('URL_FIREFLIES')

# Desabilitar o aviso de requisição insegura
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_transcripts():
    try:
        # Obter o horário atual
        current_time = datetime.now()
        print('current_time.hour', current_time.hour)
        if current_time.hour == 9:
            # Se for 9h, 15 horas antes pra buscar a partir das 18h do dia anterior
            current_time_adjusted = current_time - timedelta(hours=15)
        else:
            # Caso contrário, 3 horas antes
            current_time_adjusted = current_time - timedelta(hours=3)

        # Adicionar 3 horas ao horário atual
        current_time_plus_3 = current_time_adjusted + timedelta(hours=3)
        # Converter para o formato ISO 8601
        current_time_iso = current_time_plus_3.isoformat()
        current_time_iso = "2024-10-28T12:37:40.695001"
        print(current_time_iso)


        payload = f"{{\"query\":\"query Transcripts($fromDate: DateTime) {{ transcripts(fromDate: $fromDate) {{ title id dateString }} }}\",\"variables\":{{\"fromDate\":\"{current_time_iso}\"}}}}"
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f"Bearer {fireflies_api_key}"
        }

        response = requests.post(os.getenv('URL_FIREFLIES'), headers=headers, data=payload, verify=False)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            data = response.json()
            print('data',data)
            transcripts = data['data']["transcripts"]

            # Salvar os dados em um arquivo JSON
            with open('transcripts.json', 'w') as f:
                json.dump(transcripts, f, indent=4)
            print("Transcrições salvas em 'transcripts.json'.")

        else:
            print('Falha na requisição para o Fireflies.')

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
    except Exception as e:
        print(f"Erro geral: {e}")

# Chame essa função apenas quando quiser atualizar os dados
fetch_transcripts()

current_time.hour 17
2024-10-28T12:37:40.695001
data {'data': {'transcripts': [{'title': '[PlayZUP] Bate-papo Inicial! (Lorenzo Lessa Menegaz)', 'id': '4XPGUakkurJ0G4gH', 'dateString': '2024-10-28T17:29:04.604Z'}, {'title': '[PlayZUP] Bate-papo Inicial! (Diego Cardoso)', 'id': 'cOJq31gMFzOD5m3i', 'dateString': '2024-10-28T14:28:50.000Z'}, {'title': 'Rodrigo Gomes / Zup', 'id': 'IOwXU3rafQ1w1QdV', 'dateString': '2024-10-28T13:58:24.873Z'}, {'title': 'Webinar - Uber corporativo', 'id': 'pBzVW6Xli4AiZjut', 'dateString': '2024-10-28T12:58:46.958Z'}]}}
Transcrições salvas em 'transcripts.json'.


In [4]:
s3 = boto3.client('s3', region_name='sa-east-1')
url = os.getenv('URL_FIREFLIES')

from fpdf import FPDF

    
def lambda_handler():
    # Ler os dados salvos no arquivo JSON
    try:
        with open('transcripts.json', 'r') as f:
            transcripts = json.load(f)
    except FileNotFoundError:
        print("Arquivo 'transcripts.json' não encontrado. Execute a função 'fetch_transcripts()' primeiro.")
        return
    

    # Processar cada transcrição e gerar o PDF
    for item in transcripts:
        print('item', item)

        formatted_date = format_date(item['dateString'])
        pdf_content = f"{item['title']} - {formatted_date}\n\n"
        
        # Atualizar o pdf_content com as informações da transcrição
        pdf_content = get_sentences(pdf_content, item['id'])
        pdf_filename = f"{formatted_date}-{item['id']}.pdf"
        
        # Gerar o PDF com o conteúdo atualizado
        pdf_path = format_pdf(pdf_content, pdf_filename)

        print(f"PDF gerado em handleer depois fformat_pdf: {pdf_path}")
        
        save_pdf_s3(pdf_path, pdf_filename)

def format_pdf(content, filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Helvetica", size=12)

        # Defina a largura da célula para a largura da página menos as margens
        page_width = pdf.epw  # epw é a largura da área de conteúdo da página

        # Adiciona o conteúdo diretamente ao PDF
        pdf.multi_cell(page_width, 10, content)

        # Salva o PDF no caminho especificado
        pdf_output = f"/tmp/{filename}"
        pdf.output(pdf_output)
        print('PDF salvo em FORMAT PDF:', pdf_output)
        return pdf_output
    except Exception as e:
        print(f"Erro ao criar PDF: {e}")
        raise
  
def format_date(date_string):
    try:
        # Converter a string para um objeto datetime
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
        formatted_date = date_object.strftime("%d-%m-%Y")
        return formatted_date
    except Exception as e:
        print(f"Erro ao formatar data: {e}")
        raise

def get_sentences(pdf_content, id): 
    print(id)
    payload = f"{{\"query\":\"query Transcript($transcriptId: String!) {{ transcript(id: $transcriptId) {{ title id dateString sentences {{ index speaker_name text }} }} }}\",\"variables\":{{\"transcriptId\":\"{id}\"}}}}"
        
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f"Bearer {fireflies_api_key}"
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = json.loads(response.text)
        transcript = data['data']["transcript"]
        print('transcript', transcript)

        previous_speaker = ""
        for sentence in transcript['sentences']:
            if previous_speaker:
                if sentence['speaker_name'] == previous_speaker:
                    pdf_content = pdf_content.rstrip('\n\n') + f" {sentence['text']}\n\n"
                else:
                    pdf_content += f"{sentence['speaker_name']}: {sentence['text']}\n\n"
            previous_speaker = sentence['speaker_name']
        
        return pdf_content
    else:
        print(f"Erro na requisição: {response.status_code}")
        return pdf_content  # Retornar o conteúdo original se a requisição falhar

def save_pdf_s3(pdf_path, pdf_filename):
            # Salvar o PDF no S3
        s3 = boto3.client('s3', region_name='sa-east-1')
        bucket_name = os.getenv('BUCKET_S3')
        try:
            with open(pdf_path, "rb") as pdf_file:
                s3.put_object(Bucket=bucket_name, Key=pdf_filename, Body=pdf_file)
            print(f"PDF salvo no S3: {pdf_filename}")
        except Exception as e:
            print(f"Erro ao salvar o PDF no S3: {e}")

lambda_handler()

item {'title': '[PlayZUP] Bate-papo Inicial! (Lorenzo Lessa Menegaz)', 'id': '4XPGUakkurJ0G4gH', 'dateString': '2024-10-28T17:29:04.604Z'}
4XPGUakkurJ0G4gH
transcript {'title': '[PlayZUP] Bate-papo Inicial! (Lorenzo Lessa Menegaz)', 'id': '4XPGUakkurJ0G4gH', 'dateString': '2024-10-28T17:29:04.604Z', 'sentences': [{'index': 0, 'speaker_name': 'Lorenzo Lessa Menegaz', 'text': 'No perfil da Azupi, eu gostei bastante e, perdão, tô meio de garganta, então, provavelmente, vou tossir um pouquinho, mas, atualmente, eu trabalho numa empresa que faz, basicamente, uma coisa bem parecida que a Azupi faz, que é a parte de consulting e esses serviços, então, eu dei uma olhada no perfil da Azupi, no LinkedIn também, me identifiquei com a cultura da Azupi e conheci.'}, {'index': 1, 'speaker_name': 'Leandro Alves - Talent Acquisition', 'text': 'Legal, então vamos lá.'}, {'index': 2, 'speaker_name': 'Leandro Alves - Talent Acquisition', 'text': 'Aqui nesses 30 minutinhos nós vamos nos alinhar ali sobre 

KeyboardInterrupt: 

In [5]:

# Configurações
load_dotenv('config/.env')

api_key = os.getenv('API_KEY_FIREFLIES')

print(f"API Key: {api_key}")


API Key: 467f3c6a-e4a7-4910-9500-34c302dfa15c
